In [1]:
# https://stackoverflow.com/questions/49093729/remove-background-of-any-image-using-opencv

import json
import cv2
import numpy as np

In [2]:
with open('mergedSculptures.json') as input_file:
    data = json.load(input_file)

In [3]:
# https://stackoverflow.com/a/55590133

def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [4]:
#== Parameters           
BLUR = 41
CANNY_THRESH_1 = 15
CANNY_THRESH_2 = 40
MASK_DILATE_ITER = 19
MASK_ERODE_ITER = 23
SHARPEN_MASK = 128.0
MASK_COLOR = (1,1,1) # In BGR format
        
for e in data:
    print('Filename: ' + str(e['fileName']))
    input_path = '/Volumes/passport/resized/' + str(e['fileName'])

    #-- Read image
    # print(input_path)
    if e['fileName'] != "":
        img = cv2.imread(input_path)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        #-- Edge detection 
        edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
        edges = cv2.dilate(edges, None)
        edges = cv2.erode(edges, None)

        #-- Find contours in edges, sort by area 
        contour_info = []
        contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for c in contours:
            contour_info.append((
                c,
                cv2.isContourConvex(c),
                cv2.contourArea(c),
            ))
        contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
        max_contour = contour_info[0]

        #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
        # Mask is black, polygon is white
        mask = np.zeros(edges.shape)
        for c in contour_info:
            cv2.fillConvexPoly(mask, c[0], (255))
        #-- Smooth mask, then blur it
        mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
        mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
        mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
        mask = unsharp_mask(mask, amount=SHARPEN_MASK)
        mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

        #-- Blend masked img into MASK_COLOR background
        mask_stack  = mask_stack.astype('float32') / 255.0         
        img         = img.astype('float32') / 255.0    
        masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR)  
        masked = (masked * 255).astype('uint8')  

        #-- Get bounding Rectangle
        mask_x, mask_y , mask_width, mask_height = cv2.boundingRect(mask)
        # cv2.rectangle(masked,(x,y),(x+w,y+h),(0,255,0),2)
        original_height = img.shape[0]
        original_width = img.shape[0]
        print(mask_height < original_height)
        print(mask_width < original_width)
        nice_height = mask_y + mask_height < original_height
        nice_width = mask_x + mask_width < original_width
        nice_mask = nice_height and nice_width
        # print(h)
        # print(w)
        e['original_image_h'] = original_height
        e['original_image_w'] = original_width
        e['boundingRectHeight'] = mask_height
        e['boundingRectHeightRatio'] = original_height / mask_height
        e['boundingRectWidth'] = mask_width
        e['boundingRectWidthRatio'] = original_width / mask_width
        e['nice_mask'] = nice_mask
        e['nice_height'] = nice_height

        # Write
        output_path = '/Volumes/passport/resized_clipped/' + str(e['fileName'])
        cv2.imwrite(output_path, masked)

Filename: AW_92_10_96a.jpg
True
True
Filename: 
Filename: 34240.jpg
True
True
Filename: 34236.jpg
True
True
Filename: 34237.jpg
False
True
Filename: 148776.jpg
False
True
Filename: 25688.jpg
False
True
Filename: 25798.jpg
False
True
Filename: 25680.jpg
False
True
Filename: 25773.jpg
False
True
Filename: ADA3059.jpg
True
True
Filename: 148778.jpg
False
True
Filename: 148777.jpg
True
True
Filename: 
Filename: 158746.jpg
True
True
Filename: 34238.jpg
True
True
Filename: 232079.jpg
False
True
Filename: 34239.jpg
True
True
Filename: 148775.jpg
True
True
Filename: 
Filename: 
Filename: DP-16170-043.jpg
True
True
Filename: DP-16170-039.jpg
True
True
Filename: DP-16170-041.jpg
True
True
Filename: 156520.jpg
True
True
Filename: DP205017.jpg
True
True
Filename: 
Filename: 139181.jpg
False
True
Filename: 139182.jpg
False
True
Filename: 64.92a.jpg
True
True
Filename: 13.214.jpg
False
True
Filename: 69.219.2.jpg
True
True
Filename: 1978.511.jpg
True
True
Filename: 
Filename: 
Filename: 
Filename: 


False
True
Filename: 
Filename: 
Filename: 
Filename: 
Filename: 
Filename: sfsb29.158.731S1.jpg
False
True
Filename: DP100688.jpg
True
True
Filename: 1975_415_3_206851.jpg
True
True
Filename: 1987_142_374_O.jpg
False
False
Filename: DP252964.jpg
True
True
Filename: 1987_142_309_DT8680.jpg
True
True
Filename: DP123358.jpg
True
True
Filename: DT8567.jpg
False
True
Filename: DT241.jpg
True
True
Filename: DP206242.jpg
False
True
Filename: 64_12.JPG
False
True
Filename: 69_247.jpg
False
True
Filename: LK.1977.241_DP310109R2_25W.jpg
True
True
Filename: 1987_142_288_244935.jpg
False
True
Filename: 1987_142_287_244934.jpg
False
True
Filename: DP274258.jpg
True
True
Filename: DP123344.jpg
True
True
Filename: 1987_142_380.jpg
False
False
Filename: DT5720.jpg
True
True
Filename: DT9104.jpg
True
True
Filename: 1975_414.jpg
False
True
Filename: 1987_218_14_O.jpg
False
True
Filename: 264841.jpg
False
True
Filename: 1993_477_4.jpg
False
True
Filename: DP123374.jpg
True
True
Filename: DT8660.jpg
True

True
True
Filename: DP156771.jpg
True
True
Filename: DP156776.jpg
True
True
Filename: DP170261.jpg
True
True
Filename: 2015_300_254a_f_Group1_Burke_edit.jpg
True
False
Filename: DP248238.jpg
True
True
Filename: 2011_576_3_F_sf.jpg
True
True
Filename: DP326722.jpg
True
True
Filename: DP344318.jpg
True
False
Filename: 30736.jpg
False
False
Filename: 32154.jpg
False
True
Filename: 541.jpg
False
True
Filename: 24261.jpg
True
True
Filename: 223296.jpg
False
True
Filename: DP214413.jpg
True
True
Filename: ES4905.jpg
False
True
Filename: 30697.jpg
False
True
Filename: 30708.jpg
False
True
Filename: DP-16097-117.jpg
True
True
Filename: 17989.jpg
False
True
Filename: 32077.jpg
True
True
Filename: 11524.jpg
True
True
Filename: 36994.jpg
False
True
Filename: 
Filename: 2195.jpg
False
True
Filename: 2200.jpg
False
True
Filename: 1891.jpg
True
True
Filename: 1975.jpg
False
True
Filename: ES368.jpg
False
True
Filename: 34167.jpg
True
False
Filename: 4406.jpg
False
False
Filename: 4406.jpg
False
Fals

True
True
Filename: 161703.jpg
True
True
Filename: 160496.jpg
False
True
Filename: 
Filename: DP234091.jpg
True
True
Filename: 
Filename: DP214576.jpg
False
True
Filename: DP249454.jpg
True
True
Filename: 167142.jpg
True
True
Filename: 167819.jpg
False
True
Filename: DP-258.001.jpg
True
True
Filename: 176538.jpg
False
False
Filename: ES4787.jpg
False
False
Filename: DP249462.jpg
True
True
Filename: 
Filename: DP105180.jpg
False
False
Filename: DP104979.jpg
False
False
Filename: DP104980.jpg
False
False
Filename: DP104981.jpg
False
False
Filename: DP215256.jpg
True
True
Filename: E146243.jpg
True
True
Filename: DP-15401-067.jpg
True
True
Filename: DP-221-001.jpg
True
True
Filename: 
Filename: 201585.jpg
True
True
Filename: 201608.jpg
True
True
Filename: DT229157.jpg
True
True
Filename: DP105184.jpg
True
False
Filename: ES5344.jpg
False
False
Filename: 231291.jpg
True
True
Filename: 231292.jpg
True
True
Filename: 180753.jpg
False
False
Filename: 180753.jpg
False
False
Filename: 180753.jp

True
True
Filename: DP269779.jpg
True
True
Filename: DP157061.jpg
True
True
Filename: DP155785.jpg
True
True
Filename: DP263908.jpg
True
True
Filename: DP158168.jpg
True
True
Filename: DP158138.jpg
True
True
Filename: DP158139.jpg
True
True
Filename: DP153457.jpg
True
True
Filename: DP158140.jpg
True
True
Filename: DP155793.jpg
True
True
Filename: GR344_74.51.2571.jpg
True
True
Filename: DP155784.jpg
True
True
Filename: DP156968.jpg
True
True
Filename: DP159435.jpg
True
True
Filename: DP158141.jpg
True
True
Filename: DP214183.jpg
True
True
Filename: DP215172.jpg
True
False
Filename: DP204727.jpg
True
True
Filename: DP155786.jpg
True
True
Filename: DP206620.jpg
True
True
Filename: DP121759.jpg
True
True
Filename: DP263914.jpg
True
True
Filename: DP159436.jpg
True
True
Filename: DP208992.jpg
True
True
Filename: DP263915.jpg
True
True
Filename: DP158143.jpg
True
True
Filename: DP202648.jpg
True
True
Filename: DP159437.jpg
True
True
Filename: DP212765.jpg
True
True
Filename: DP157064.jpg
T

True
True
Filename: 78072.jpg
False
True
Filename: 78074.jpg
False
True
Filename: DP333739.jpg
True
False
Filename: DP122081.jpg
True
True
Filename: DP123886.jpg
True
True
Filename: DP114994.jpg
True
False
Filename: DP138715.jpg
True
True
Filename: DP331288.jpg
True
True
Filename: DP331293.jpg
True
True
Filename: 72387.jpg
False
True
Filename: 79714.jpg
False
False
Filename: DT259949.jpg
True
True
Filename: DP119249.jpg
True
True
Filename: DT247660.jpg
True
True
Filename: DP345017.jpg
True
True
Filename: DP202855.jpg
True
True
Filename: DP277772.jpg
True
True
Filename: DT11989.jpg
True
True
Filename: DP138727.jpg
True
True
Filename: DP333080.jpg
True
True
Filename: DT259950.jpg
False
True
Filename: DP206632.jpg
True
True
Filename: DP119970.jpg
True
True
Filename: DP200682.jpg
True
True
Filename: DP119971.jpg
True
True
Filename: DP203145.jpg
True
True
Filename: DP206633.jpg
True
True
Filename: DP200683.jpg
True
True
Filename: DP116928.jpg
True
False
Filename: DP116944.jpg
True
True
File

True
True
Filename: 1979.206.358.jpg
True
True
Filename: DP341932.jpg
True
True
Filename: 1979.206.366_a.JPG
True
True
Filename: DP-17678-001.jpg
True
True
Filename: 
Filename: 1979.206.417.jpg
True
True
Filename: 1979.206.418.JPG
True
True
Filename: vs1979_206_430.jpg
False
True
Filename: DP261003.jpg
True
True
Filename: 1979.206.530.jpg
False
True
Filename: vs1979_206_565.jpg
False
True
Filename: 1979.206.578.jpg
True
True
Filename: vs1979_206_699.jpg
False
True
Filename: 1979.206.702.jpg
True
True
Filename: DP-16189-015.jpg
True
True
Filename: DP350139.jpg
True
True
Filename: 1979.206.780.jpg
True
True
Filename: 1979.206.870.JPG
False
True
Filename: 1979.206.871.JPG
False
True
Filename: 1979.206.1033_b.jpg
False
True
Filename: 1979.206.926.jpg
True
True
Filename: 1979.206.933_a.jpg
True
True
Filename: DT9963.jpg
False
True
Filename: 1979.206.949.jpg
False
True
Filename: 1979.206.962.JPG
True
True
Filename: 1979.206.963.JPG
False
True
Filename: 1979.206.964.JPG
False
True
Filename: D

KeyError: 'fileName'

In [5]:
for e in data: 
     print('Bounding Rectangle Pixel Height: ' + str(e['boundingRectHeight']))

Bounding Rectangle Pixel Height: 307


KeyError: 'boundingRectHeight'

In [6]:
with open('full-boundingRectClipped.json', 'w') as output_file:
    json.dump(data, output_file)